# GTN官方代码测试及说明

### 导入相关库文件

In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from model import GTN
import pdb
import pickle
import argparse
from utils import f1_score

### 配置相关参数

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--dataset', type=str, default='DBLP',
                    help='Dataset')
parser.add_argument('--epoch', type=int, default=40,
                    help='Training Epochs')
parser.add_argument('--node_dim', type=int, default=64,
                    help='Node dimension')
parser.add_argument('--num_channels', type=int, default=2,
                    help='number of channels')
parser.add_argument('--lr', type=float, default=0.005,
                    help='learning rate')
parser.add_argument('--weight_decay', type=float, default=0.001,
                    help='l2 reg')
parser.add_argument('--num_layers', type=int, default=2,
                    help='number of layer')
parser.add_argument('--norm', type=str, default='true',
                    help='normalization')
parser.add_argument('--adaptive_lr', type=str, default='false',
                    help='adaptive learning rate')

#args = parser.parse_args()                                               # pychram 中使用
args = parser.parse_known_args()[0]                                       # jupyter 中使用
print(args)

Namespace(dataset='DBLP', epoch=40, node_dim=64, num_channels=2, lr=0.005, weight_decay=0.001, num_layers=2, norm='true', adaptive_lr='false')


In [3]:
epochs = args.epoch
node_dim = args.node_dim
num_channels = args.num_channels
lr = args.lr
weight_decay = args.weight_decay
num_layers = args.num_layers
norm = args.norm
adaptive_lr = args.adaptive_lr

### 加载数据

In [4]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
with open('../data/'+args.dataset+'/node_features.pkl','rb') as f:
    node_features = pickle.load(f)
with open('../data/'+args.dataset+'/edges.pkl','rb') as f:
    edges = pickle.load(f)
with open('../data/'+args.dataset+'/labels.pkl','rb') as f:
    labels = pickle.load(f)
num_nodes = edges[0].shape[0]

In [11]:
print(node_features.shape)
print(edges[0].shape)
print(len(labels))
print(labels[0])

(18405, 334)
(18405, 18405)
3
[[0, 1], [1, 3], [2, 0], [3, 3], [4, 0], [5, 0], [6, 3], [7, 3], [8, 2], [9, 0], [10, 2], [11, 0], [12, 0], [13, 0], [14, 1], [15, 2], [16, 3], [17, 0], [18, 2], [19, 3], [20, 0], [21, 1], [22, 2], [23, 2], [24, 2], [25, 2], [26, 2], [27, 2], [28, 0], [29, 1], [30, 0], [31, 0], [32, 1], [33, 2], [34, 2], [35, 3], [36, 3], [37, 2], [38, 3], [39, 0], [40, 3], [41, 2], [42, 1], [43, 2], [44, 1], [45, 0], [46, 0], [47, 1], [48, 0], [49, 1], [50, 1], [51, 2], [52, 3], [53, 3], [54, 2], [55, 2], [56, 3], [57, 1], [58, 3], [59, 2], [60, 0], [61, 1], [62, 1], [63, 3], [64, 3], [65, 2], [66, 2], [67, 2], [68, 1], [69, 1], [70, 2], [71, 3], [72, 0], [73, 0], [74, 0], [75, 1], [76, 0], [77, 3], [78, 0], [79, 0], [80, 1], [81, 0], [82, 1], [83, 2], [84, 2], [85, 3], [86, 0], [87, 3], [88, 0], [89, 0], [90, 2], [91, 3], [92, 2], [93, 0], [94, 3], [95, 0], [96, 0], [97, 0], [98, 0], [99, 3], [100, 2], [101, 0], [102, 2], [103, 3], [104, 0], [105, 2], [106, 2], [107, 3],

### 构造邻接矩阵

**对图的邻接矩阵进行处理，将其转换为PyTorch张量，并将其拼接在一起:**

1. 遍历edges中的每个邻接矩阵edge,其中i表示当前遍历的邻接矩阵索引。
2. 如果i为0，则将edge转换为PyTorch张量，并将其unsqueeze到最后一维以便在之后进行拼接。
3. 如果i不为0，则将当前邻接矩阵edge转换为PyTorch张量，并将其unsqueeze到最后一维，然后将结果与之前的邻接矩阵A进行cat操作，将它们拼接在一起。
4. 最后，将torch.eye(num nodes)(一个单位矩阵)转换为PyTorch张量，并将其unsqueeze到最后一维，然后将结果与A进行cat操作，将它们拼接在一起。这样就得到了最终的邻接矩阵。

In [13]:
for i,edge in enumerate(edges):
    if i ==0:
        A = torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)
    else:
        A = torch.cat([A,torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)
A = torch.cat([A,torch.eye(num_nodes).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)

In [14]:
A.shape

torch.Size([18405, 18405, 5])

### 将训练集、验证集和测试集数据转化为LongTensor格式

In [15]:
node_features = torch.from_numpy(node_features).type(torch.FloatTensor)
train_node = torch.from_numpy(np.array(labels[0])[:,0]).type(torch.LongTensor)
train_target = torch.from_numpy(np.array(labels[0])[:,1]).type(torch.LongTensor)
valid_node = torch.from_numpy(np.array(labels[1])[:,0]).type(torch.LongTensor)
valid_target = torch.from_numpy(np.array(labels[1])[:,1]).type(torch.LongTensor)
test_node = torch.from_numpy(np.array(labels[2])[:,0]).type(torch.LongTensor)
test_target = torch.from_numpy(np.array(labels[2])[:,1]).type(torch.LongTensor)

In [16]:
print(node_features.shape)
print(train_node.shape)
print(train_target.shape)
print(valid_node.shape)
print(valid_target.shape)
print(test_node.shape)
print(test_target.shape)
print(A.shape)

torch.Size([18405, 334])
torch.Size([800])
torch.Size([800])
torch.Size([400])
torch.Size([400])
torch.Size([2857])
torch.Size([2857])
torch.Size([18405, 18405, 5])


### 最终的类别数

In [12]:
num_classes = torch.max(train_target).item()+1
final_f1 = 0
num_classes

4

### 模型训练

In [13]:
for l in range(1):
    model = GTN(num_edge=A.shape[-1], # 图中不同类型的边的类别数，从邻接矩阵A的最后一维中获取
                        num_channels=num_channels, # 图卷积层的通道数
                        w_in = node_features.shape[1], # 输入特征的维度，从节点特征node_features.shape[1]中获取
                        w_out = node_dim, # 隐藏层输出的特征维度
                        num_class=num_classes, # 分类任务中的类别数
                        num_layers=num_layers, # 图卷积层的数量
                        norm=norm) # 是否对邻接矩阵进行归一化
    
    if adaptive_lr == 'false': # 学习率是否为自适应
        optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
    else:
        optimizer = torch.optim.Adam([{'params':model.weight},
                                    {'params':model.linear1.parameters()},
                                    {'params':model.linear2.parameters()},
                                    {"params":model.layers.parameters(), "lr":0.5}
                                    ], lr=0.005, weight_decay=0.001)
    loss = nn.CrossEntropyLoss() # 用于多分类任务的标准交叉熵损失函数

    # Train & Valid & Test
    best_val_loss = 10000
    best_test_loss = 10000
    best_train_loss = 10000
    best_train_f1 = 0
    best_val_f1 = 0
    best_test_f1 = 0
    
    # for i in range(epochs):
    for i in range(1):
        for param_group in optimizer.param_groups:
            if param_group['lr'] > 0.005:
                param_group['lr'] = param_group['lr'] * 0.9  # 学习率衰减
        print('Epoch:  ',i+1)

        model.zero_grad()   # 梯度归零
        model.train()       # 声明模型训练
        loss,y_train,Ws = model(A, node_features, train_node, train_target) # 损失值和结果
        train_f1 = torch.mean(f1_score(torch.argmax(y_train.detach(),dim=1), train_target, num_classes=num_classes)).cpu().numpy() # 训练集的F1得分
        print('Train - Loss: {}, Macro_F1: {}'.format(loss.detach().cpu().numpy(), train_f1))

        loss.backward() # 反向传播
        optimizer.step() # 梯度更新

        # Valid
        model.eval() # 声明模型测试
        with torch.no_grad(): # 无梯度声明
            val_loss, y_valid,_ = model.forward(A, node_features, valid_node, valid_target)
            val_f1 = torch.mean(f1_score(torch.argmax(y_valid,dim=1), valid_target, num_classes=num_classes)).cpu().numpy() # 验证集的F1得分
            print('Valid - Loss: {}, Macro_F1: {}'.format(val_loss.detach().cpu().numpy(), val_f1))
            test_loss, y_test,W = model.forward(A, node_features, test_node, test_target)
            test_f1 = torch.mean(f1_score(torch.argmax(y_test,dim=1), test_target, num_classes=num_classes)).cpu().numpy() # 测试集的F1得分
            print('Test - Loss: {}, Macro_F1: {}\n'.format(test_loss.detach().cpu().numpy(), test_f1))
        
        if val_f1 > best_val_f1:
            best_val_loss = val_loss.detach().cpu().numpy()
            best_test_loss = test_loss.detach().cpu().numpy()
            best_train_loss = loss.detach().cpu().numpy()
            best_train_f1 = train_f1
            best_val_f1 = val_f1
            best_test_f1 = test_f1 
            
    print('---------------Best Results--------------------')
    print('Train - Loss: {}, Macro_F1: {}'.format(best_train_loss, best_train_f1))
    print('Valid - Loss: {}, Macro_F1: {}'.format(best_val_loss, best_val_f1))
    print('Test - Loss: {}, Macro_F1: {}'.format(best_test_loss, best_test_f1))
    final_f1 += best_test_f1

Epoch:   1
Train - Loss: 1.390804409980774, Macro_F1: 0.11158082634210587
Valid - Loss: 1.3697787523269653, Macro_F1: 0.2811485528945923
Test - Loss: 1.3840534687042236, Macro_F1: 0.24715301394462585

---------------Best Results--------------------
Train - Loss: 1.390804409980774, Macro_F1: 0.11158082634210587
Valid - Loss: 1.3697787523269653, Macro_F1: 0.2811485528945923
Test - Loss: 1.3840534687042236, Macro_F1: 0.24715301394462585
